In [19]:
import pandas as pd
import matplotlib.pyplot as plt
from handlers.database import get_crime_relationship_statistics_from_mongo

In [20]:
relationship_statistics = get_crime_relationship_statistics_from_mongo()
relationship_statistics = relationship_statistics["statistics"]
relationships = relationship_statistics["assaults"]
relationships_normalized = relationship_statistics["assaults_normalized"]
relationships_without_assaults = relationship_statistics["without_assaults"]
relationships_without_assaults_normalized = relationship_statistics["without_assaults_normalized"]

relationship_df = pd.DataFrame(relationships)
relationship_df_normalized = pd.DataFrame(relationships_normalized)
relationship_df_without_assaults = pd.DataFrame(relationships_without_assaults)
relationship_df_without_assaults_normalized = pd.DataFrame(relationships_without_assaults_normalized)

In [21]:
def move_legend_outside_plot(ax):
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

def add_relationship_labels(ax):
    ax.set_title("Proportion of Criminal Acts for each Relationship Type")
    ax.set_xlabel("Proportion")
    ax.set_ylabel("Relationship Type")

In [22]:
import seaborn as sns

def plot_swagger_df(df, title, normalized=False):
    """
    Create a sleek horizontal stacked bar chart using seaborn and matplotlib.

    Parameters:
    - df: pd.DataFrame, the data to plot (columns as categories, rows as index)
    - title: str, the title of the plot
    - normalized: bool, whether to normalize rows to sum to 1
    """
    # Normalize rows if needed
    if normalized:
        df = df.div(df.sum(axis=1), axis=0)
    
    # Set a sleek seaborn style
    sns.set_theme(style="whitegrid", palette="pastel")
    
    # Create the figure and axis
    fig, ax = plt.subplots(figsize=(12, 7))
    
    # Get a vibrant color palette for the bars
    colors = sns.color_palette("husl", n_colors=df.shape[1])
    
    # Plot each column as a stacked horizontal bar
    bottom = None
    for i, column in enumerate(df.columns):
        sns.barplot(
            x=df[column],
            y=df.index,
            color=colors[i],
            label=column,
            orient='h',
            ax=ax,
            left=bottom
        )
        bottom = df.iloc[:, :i+1].sum(axis=1) if bottom is None else bottom + df[column]
    
    # Add title with a larger, bold font
    ax.set_title(title, fontsize=18, weight='bold', pad=20)
    
    # Move the legend outside the plot
    ax.legend(title="Categories", bbox_to_anchor=(1.05, 1), loc='upper left', frameon=True)
    
    # Add gridlines and make them more subtle
    ax.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.6)
    
    # Customize y-axis and x-axis labels
    ax.set_ylabel("", fontsize=14)
    ax.set_xlabel("Proportion" if normalized else "Value", fontsize=14)
    
    # Tighten layout and show the plot
    plt.tight_layout()
    plt.show()

In [23]:
import seaborn as sns
def plot_df(df, title):
    ax = df.plot(kind='barh', stacked=True, figsize=(10, 6), title=title)
    move_legend_outside_plot(ax)
    add_relationship_labels(ax)
    plt.show()

In [ ]:
plot_swagger_df(relationship_df, "Proportion of Criminal Acts for each Relationship Type", normalized=False)
plot_swagger_df(relationship_df_normalized, "Proportion of Criminal Acts for each Relationship Type (Normalized)", normalized=True)
plot_swagger_df(relationship_df_without_assaults, "Proportion of Criminal Acts for each Relationship Type (Without Assaults)", normalized=False)
plot_swagger_df(relationship_df_without_assaults_normalized, "Proportion of Criminal Acts for each Relationship Type (Without Assaults, Normalized)", normalized=True)